In [1]:
import wandb

import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Flatten, Conv2D, MaxPool2D, TimeDistributed, LSTM,Conv1D,MaxPool1D,Bidirectional
from tensorflow.keras.models import Model

from TranscriptionDataset.TensorflowLoader import MusicMetaSequence

print("TensorFlow version:", tf.__version__)

2023-06-19 21:18:42.644239: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-19 21:18:43.440576: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


TensorFlow version: 2.12.0


In [ ]:
wandb.init(project='TFArrangement')

In [2]:
trainSequence = MusicMetaSequence("Trainsets/S_Tier_Mel.hdf5")
# trainSequence[0][0].shape,trainSequence[0][1].shape

In [3]:
input_layer = Input(shape=(None,256),batch_size=1)
x = (Conv1D(data_format="channels_last",filters=128, kernel_size=5, padding="same", activation="relu"))(input_layer)
x = (MaxPool1D(pool_size=2))(x)
x = (Conv1D(data_format="channels_last",filters=64, kernel_size=5, padding="same", activation="relu"))(x)
x = (MaxPool1D(pool_size=2))(x)
x = (Conv1D(data_format="channels_last",filters=8, kernel_size=5, padding="same", activation="relu"))(x)
x = (MaxPool1D(pool_size=2))(x)
x = Bidirectional(LSTM(128))(x)
x = Dense(128,activation="relu")(x)
outputs = []
for i in range(6):
    outputs.append(Dense(1,name=f'output_{i}',activation="sigmoid")(x))

model = Model(inputs=input_layer, outputs=outputs)

print(model.summary())

2023-06-19 21:18:50.218437: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-19 21:18:50.221840: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-19 21:18:50.221894: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-19 21:18:50.225166: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-19 21:18:50.225248: I tensorflow/compile

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(1, None, 256)]     0           []                               
                                                                                                  
 conv1d (Conv1D)                (1, None, 128)       163968      ['input_1[0][0]']                
                                                                                                  
 max_pooling1d (MaxPooling1D)   (1, None, 128)       0           ['conv1d[0][0]']                 
                                                                                                  
 conv1d_1 (Conv1D)              (1, None, 64)        41024       ['max_pooling1d[0][0]']          
                                                                                              

In [4]:
optimizer = tf.keras.optimizers.AdamW(learning_rate=0.001)
loss = {}
metrics = {}
for i in range(6):
    loss[f'output_{i}'] = tf.keras.losses.BinaryFocalCrossentropy()
    metrics[f"output_{i}"] = tf.keras.metrics.Accuracy()
model.compile(optimizer=optimizer,loss=loss,metrics=metrics)


In [5]:
model.fit(trainSequence,epochs=10,verbose=1)

Epoch 1/10


2023-06-19 21:18:53.916944: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-19 21:18:54.267417: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-19 21:18:54.270626: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_gra

1269/1269 [==============================] - 613s 467ms/step - loss: 0.2411 - output_0_loss: 0.0218 - output_1_loss: 0.0994 - output_2_loss: 0.0040 - output_3_loss: 0.0559 - output_4_loss: 0.0562 - output_5_loss: 0.0038 - output_0_accuracy: 0.0000e+00 - output_1_accuracy: 0.0000e+00 - output_2_accuracy: 0.0000e+00 - output_3_accuracy: 0.0000e+00 - output_4_accuracy: 0.0000e+00 - output_5_accuracy: 0.0000e+00
Epoch 2/10
1269/1269 [==============================] - 213s 168ms/step - loss: 0.2244 - output_0_loss: 0.0206 - output_1_loss: 0.0966 - output_2_loss: 0.0032 - output_3_loss: 0.0517 - output_4_loss: 0.0494 - output_5_loss: 0.0029 - output_0_accuracy: 0.0000e+00 - output_1_accuracy: 0.0000e+00 - output_2_accuracy: 0.0000e+00 - output_3_accuracy: 0.0000e+00 - output_4_accuracy: 0.0000e+00 - output_5_accuracy: 0.0000e+00
Epoch 3/10
 515/1269 [===========>..................] - ETA: 1:56 - loss: 0.2171 - output_0_loss: 0.0255 - output_1_loss: 0.0965 - output_2_loss: 2.4968e-04 - output


KeyboardInterrupt

